In [9]:
import numpy as np
import pandas as pd
import sklearn.preprocessing as preprocessing


In [12]:
df = pd.read_csv(
    "datas/EURUSD_M5.csv",
    sep="\t",
    nrows=2**16,
    header=0,
    dtype={
        "open": np.float64,
        "close": np.float64,
        "tickvol": np.float64,
        "vol": np.float64,
    },
    parse_dates=[[0,1]],
    index_col=[0],
    names=["date","time","open","high","low","close","tickvol","vol","spread"])

In [13]:
prices = np.nan_to_num(np.array(df["open"]), posinf=0, neginf=0)
prices_diff = np.diff(prices[: -64])
x = np.reshape(prices_diff[: -64], (-1, 1))

In [18]:
scaler = preprocessing.RobustScaler()
scaler.fit(x)
x_scaled = scaler.transform(x)
print(scaler.center_, scaler.scale_)
print(np.min(x_scaled), np.max(x_scaled))
x_scaled[:64]

[0.] [0.0004]
-24.250000000002775 48.00000000000527


array([[ 1.5 ],
       [-0.25],
       [ 0.25],
       [ 0.75],
       [ 1.25],
       [ 0.  ],
       [-0.25],
       [-2.  ],
       [ 2.75],
       [ 1.25],
       [ 8.75],
       [-0.25],
       [-0.75],
       [-0.5 ],
       [-0.5 ],
       [ 1.  ],
       [ 1.75],
       [-0.25],
       [ 0.5 ],
       [ 0.5 ],
       [ 1.25],
       [-1.  ],
       [ 1.  ],
       [-1.25],
       [-0.25],
       [ 0.25],
       [ 1.75],
       [-1.25],
       [ 1.25],
       [-0.5 ],
       [ 0.5 ],
       [ 0.  ],
       [ 1.75],
       [ 1.5 ],
       [ 3.75],
       [ 4.5 ],
       [-2.  ],
       [ 2.  ],
       [-0.75],
       [ 0.  ],
       [-1.25],
       [ 0.  ],
       [-0.5 ],
       [ 0.25],
       [-0.5 ],
       [-0.5 ],
       [-0.5 ],
       [-0.25],
       [ 0.25],
       [-1.5 ],
       [ 0.5 ],
       [ 0.  ],
       [-0.75],
       [ 0.25],
       [ 1.  ],
       [-1.  ],
       [ 0.  ],
       [ 0.  ],
       [ 1.  ],
       [-1.  ],
       [ 0.25],
       [ 0.75],
       [

In [44]:
print(len(df.index))
df = df.asfreq(freq='5min')
df = df.fillna(method='ffill')
print(len(df.index))

39314
54935
